In [1]:
from PIL import Image
import h5py
import anubisPlotUtils as anPlot
import json
import numpy as np
import os
import hist as hi
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
matplotlib.use('TkAgg')  # or 'Qt5Agg', 'GTK3Agg', etc.
import mplhep as hep
hep.style.use([hep.style.ATLAS])
import sys
import AnalysisToolAnubis as AT
import pandas as pd
import tkinter as tk
from tkinter import ttk
from pandasgui import show


ERROR:root:Cannot activate multiple GUI eventloops


In [2]:
#Importing the data
thisData = AT.importDatafile('C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubisData\\60sRun_24_3_4.h5')


In [3]:
mapping = {
    0: {
        (0, 31): ('rpc3', 'eta'),
        (32, 63): ('rpc3', 'phi1'),
        (64, 95): ('rpc3', 'phi2'),
        (96, 127): ('rpc2', 'eta'),
    },
    1: {
        (0, 31): ('rpc2', 'phi1'),
        (32, 63): ('rpc2', 'phi2'),
        (64, 95): ('rpc1', 'eta'),
        (96, 127): ('rpc1', 'phi1'),
    },
    2: {
        (0, 31): ('rpc1', 'phi2'),
        (32, 63): ('rpc4', 'eta'),
        (64, 95): ('rpc4', 'phi1'),
        (96, 127): ('rpc4', 'phi2'),
    },
    3: {
        (0, 31): ('rpc6', 'eta'),
        (32, 63): ('rpc6', 'phi1'),
        (64, 95): ('rpc6', 'phi2'),
        (96, 127): ('rpc5', 'eta'),
    },
    4: {
        (0, 31): ('rpc5', 'phi1'),
        (32, 63): ('rpc5', 'phi2'),
    },
}

In [4]:
def assign_to_coordinates(tdc, channel, mapping):
    for channel_range, (rpc, side) in mapping.get(tdc, {}).items():
        if channel_range[0] <= channel <= channel_range[1]:
            offset = channel - channel_range[0]
            if side == 'eta':
                return [rpc, 'eta', [0, offset]]
            elif side in ['phi1', 'phi2']:
                return [rpc, 'phi', [offset, 0]]
    return ['Null', 'Null', [tdc, channel]]  # If no match found

In [6]:
def remake_data(Data, mapping):
    full_data = []
    for tdc in range(0, len(Data)):
        for events in range(0, len(Data[tdc])): 
            for hits in Data[tdc][events]:
                channel = (hits>>24)&0x7f
                coord = assign_to_coordinates(tdc, channel, mapping)
                event_time = hits&0xfffff
                
            
                
                full_data.append({
                    'tdc': tdc,
                    'event number': events,
                    'channel':channel,
                    'rpc number': coord[0],
                    'strip direction': coord[1],
                    'hit coordinate': coord[2],
                    'time': event_time
                })
    df = pd.DataFrame(full_data)
        
    return df
    

In [7]:
df = remake_data(thisData, mapping)


In [21]:
show(df)


PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


The instruction was unclear, what does it meant by three coincidence required in the email? possibly three different RPC hit required for the trigger to record... The reason of the event time being mostly around 200 arbitrary value, is probably the time takes for the computer to react and record the correcponding time, I won't trust that value to be the exact time, but it will be interesting to see if there are correlation between those time and rpc number

In [16]:
event_rpc_dict = {}

for event in df['event number'].unique():
    rpcs_involved = df.loc[df['event number'] == event, 'rpc number'].unique()
    event_rpc_dict[event] = rpcs_involved

events_with_less_than_3_rpcs = 0

for event, rpcs in event_rpc_dict.items():
    if len(rpcs) < 3:
        events_with_less_than_3_rpcs += 1

print(events_with_less_than_3_rpcs)

8


In [19]:
event_counts = df['event number'].value_counts()

events_less_than_3_occurrences = event_counts[event_counts < 3]

num_events_less_than_3 = len(events_less_than_3_occurrences)

print(num_events_less_than_3)

0


Hence it might not have been between rpcs, try if it is between number of simulateous events
Well, that is self explainatary, and since we have 19473 entries, each event contains roughly 30.19 entries, didn't expect to be this tbh

Well it doesn't stop data analysis
total 576 strips, 19473 interaction recorded, if all through, this is about 0.56 Hz of dark counts, which is slightly lower than the expected 1Hz.

Checking if there are any broken channels (or strips)

In [52]:
all_combinations = []
for tdc in range(5):  # tdc0 to tdc4
    tdcnumber = tdc
    max_channel = 63 if tdc == 4 else 127
    for channel in range(max_channel + 1):
        all_combinations.append((tdc, channel))

all_combinations_df = pd.DataFrame(all_combinations, columns=['tdc', 'channel'])
missing_combinations = pd.merge(all_combinations_df, df, on=['tdc', 'channel'], how='left', indicator=True)
missing_combinations = missing_combinations[missing_combinations['_merge'] == 'left_only']
missing_combinations = missing_combinations[['tdc', 'channel']]
show(missing_combinations)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


tdc0 channel 31
tdc0 channel 107
tdc0 channel 111
tdc2 channel 41
tdc3 channel 51
These never fired, for a 30 second run and if the dark count is stochastic with poisson distribution, it is very unlikely this will happen, hence they are probably dead...

In [29]:
#slightly different way to store data, but does the same thing, it goes like rpc - event - strip
etaHits, phiHits = AT.divideHitCountsByRPC(thisData)

[[[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0,

In [ ]:
def countCoincidencesEvents(primRpc,SecRpc, winSize=5, minHit=3, maxHit=10):
    coincArray = [0 for channel in range(len(primRpc[0]))]
    for idx, event in enumerate(primRpc):
        for channel in range(len(event)):
            nHits = 0
            for itr in range(channel-int(winSize/2.), channel+int(winSize/2.)+1):
                if itr>=0 and itr<len(event):
                    if event[channel]>0:
                        nHits = nHits+1
            if nHits>=minHit and sum(event)<=maxHit:
                if sum(SecRpc[idx])>=minHit and sum(SecRpc[idx])<=maxHit:
                    coincArray[channel]=coincArray[channel]+1
    return coincArray

In [48]:
coincidence = {}
#The countCoincidences defined a window sieze, in which it says if the number of events within this window size is more than 3 and less than 10 hits occured per event (a cluster), 
# and if the second strip also had a a total event more than 3 and less than 10, 
# then it is recorded as a successful coincidence
for i in range(5):
    result_key = f'etaHits{i}_phiHits{i}' 
    coincidence[result_key] = AT.countCoincidences(etaHits[i], phiHits[i])
    coincidence[result_key] = AT.countCoincidences(phiHits[i], etaHits[i])
#This is useful as it gives an idea how many successful hits were observed at whatever places
print(coincidence)

{'etaHits0_phiHits0': [23, 1, 2, 2, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 4, 5, 8, 6, 2, 7, 7, 4, 8, 11, 7, 4, 4, 6, 8, 6, 3, 4, 6, 7, 9, 15, 15, 11, 9, 6, 6, 2, 2, 0, 6, 2, 7, 6, 6, 7, 4, 3, 7, 4, 4, 5, 5, 3, 5, 5, 6, 6], 'etaHits1_phiHits1': [4, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 2, 5, 4, 1, 6, 6, 4, 5, 5, 6, 9, 7, 7, 9, 5, 5, 2, 2, 1, 0, 2, 1, 2, 3, 3, 7, 7, 2, 3, 3, 3, 2, 1, 4, 7, 3, 11, 10, 9, 5, 4, 6, 10, 4, 9, 7, 8, 8, 7, 12, 11, 7], 'etaHits2_phiHits2': [24, 5, 5, 4, 4, 2, 5, 4, 6, 7, 8, 9, 6, 6, 5, 7, 10, 9, 6, 6, 3, 5, 5, 5, 5, 5, 6, 4, 3, 3, 6, 4, 8, 5, 7, 6, 7, 9, 5, 13, 8, 9, 10, 8, 2, 2, 6, 3, 12, 8, 9, 7, 4, 3, 6, 5, 11, 6, 9, 6, 6, 6, 8, 5], 'etaHits3_phiHits3': [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 3, 2, 1, 0, 1, 1, 2, 2, 2, 2, 0, 0, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0], 'etaHits4_phiHits4': [9, 3, 4, 3, 4, 3, 2, 2, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 3, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 2, 3,

Looking first at 1D data for the strippes, without any time associated with it (treating events as independant discreet series)
we can quantify a clustering metric

In [66]:
def calculate_cluster_metrics(event, minHit=3, maxHit=10):
    cluster_sizes = []
    cluster_locations = []
    current_cluster_size = 0
    start_index = None  
    
    for index, i in enumerate(event):
        if i != 0:
            current_cluster_size += i
            if start_index is None:
                start_index = index  
        elif i == 0 and current_cluster_size >= minHit and current_cluster_size < maxHit:
            # Calculate the midpoint of the cluster
            cluster_midpoint = (start_index + index - 1) / 2.0
            cluster_locations.append(cluster_midpoint)
            cluster_sizes.append(current_cluster_size)
            current_cluster_size = 0
            start_index = None 
        elif i == 0 and (current_cluster_size < minHit or current_cluster_size >= maxHit):
            current_cluster_size = 0
            start_index = None 
    
    if current_cluster_size >= minHit and current_cluster_size < maxHit:
        cluster_midpoint = (start_index + len(event) - 1) / 2.0
        cluster_locations.append(cluster_midpoint)
        cluster_sizes.append(current_cluster_size)
    
    return cluster_locations, cluster_sizes


In [67]:
columns = ['RPC Number', 'Event Number', 'cluster_locations', 'Cluster_size']
dfclus = pd.DataFrame(columns=columns)

for i, rpc_data in enumerate(etaHits):
    for j, event in enumerate(rpc_data):
        cluster_locations, cluster_sizes = calculate_cluster_metrics(event)
        for _ in range(len(cluster_sizes)):
            
            dfclus = dfclus.append({'RPC Number': i, 'Event Number': j, 'cluster_locations': cluster_locations[_], 'Cluster_size': cluster_sizes[_]}, ignore_index=True)

C:\Users\Peter\AppData\Local\Temp\ipykernel_21012\3471798240.py:9: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Peter\AppData\Local\Temp\ipykernel_21012\3471798240.py:9: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Peter\AppData\Local\Temp\ipykernel_21012\3471798240.py:9: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Peter\AppData\Local\Temp\ipykernel_21012\3471798240.py:9: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Peter\AppData\Local\Temp\ipykernel_21012\3471798240.py:9: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat 

In [68]:
show(dfclus)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI
